In [634]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
import math

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

from sklearn.metrics import log_loss
from sklearn.impute import SimpleImputer

from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

import pickle

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [635]:
df = pd.read_csv( "/home/bautista/Datos/Machine-Learning-Datos/FeatureEngineering/df_time_series.csv" )
df = df.drop(columns = {'Unnamed: 0', 'Delivery_Year', 'Delivery_Quarter'})
df

,Region,Total_Amount,TRF,Opportunity_Owner,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
0,EMEA,13.271783,1,Other,0,1,0,13.892824,13.271783,NaN,17.330865
1,EMEA,12.753460,1,Other,1,1,0,13.912927,12.753460,13.271783,17.306318
2,EMEA,17.332958,80,Other,1,1,0,13.895475,17.332958,12.753460,18.011205
3,APAC,13.386359,2,Other,0,0,0,14.012659,13.386359,17.332958,0.000000
4,APAC,11.377793,0,Person_Name_11,1,1,1,14.255779,12.723426,NaN,17.323551
...,...,...,...,...,...,...,...,...,...,...,...
16767,Americas,14.477339,6,Person_Name_8,1,1,1,14.621714,16.483576,16.318135,17.377829
16768,Americas,14.033311,4,Person_Name_8,1,1,1,14.621714,16.483576,16.318135,17.377829
16769,Americas,14.477339,6,Person_Name_8,1,1,1,14.621714,16.483576,16.318135,17.377829
16770,Americas,18.017027,195,Person_Name_8,1,1,0,14.621714,16.483576,16.318135,17.377829


In [636]:
df = df.fillna(0)

In [637]:
df.describe()

,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
count,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000,16772.000000
mean,10.470044,2.281660,0.585440,0.807834,0.568030,13.255775,12.244222,11.541450,17.309752
std,3.308478,12.084678,0.492661,0.394014,0.495365,1.329157,2.325201,3.965941,0.148853
min,-4.605170,0.000000,0.000000,0.000000,0.000000,11.058881,3.806662,0.000000,0.000000
25%,8.465995,0.000000,0.000000,1.000000,0.000000,11.464494,10.971975,10.113876,17.303476
50%,11.253377,0.000000,1.000000,1.000000,1.000000,13.892824,12.847837,12.894433,17.313452
75%,12.582682,1.000000,1.000000,1.000000,1.000000,14.012659,14.000724,14.075266,17.319557
max,19.568083,500.000000,1.000000,1.000000,1.000000,15.245204,19.568083,17.528668,19.542568


# Encode

In [638]:
Mean_encoded_subject_region = df.groupby(['Region'])['Decision'].mean().to_dict() 

df['Region'] =  df['Region'].map(Mean_encoded_subject_region)

#Mean_encoded_subject_quarter = df.groupby(['Delivery_Quarter'])['Decision'].mean().to_dict() 

#df['Delivery_Quarter'] =  df['Delivery_Quarter'].map(Mean_encoded_subject_quarter)

Mean_encoded_subject_owner = df.groupby(['Opportunity_Owner'])['Decision'].mean().to_dict() 

df['Opportunity_Owner'] =  df['Opportunity_Owner'].map(Mean_encoded_subject_owner)

# Split

In [639]:
X_test = df.tail(math.floor(df.shape[0]*0.20)).drop(columns = {'Decision'})
y_test = df.tail(math.floor(df.shape[0]*0.20))['Decision']
X_train = df.head(round(df.shape[0]*0.80)).drop(columns = {'Decision'})
y_train = df.head(round(df.shape[0]*0.80))['Decision']
X_train

,Region,Total_Amount,TRF,Opportunity_Owner,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
0,0.532241,13.271783,1,0.000000,0,1,13.892824,13.271783,0.000000,17.330865
1,0.532241,12.753460,1,0.000000,1,1,13.912927,12.753460,13.271783,17.306318
2,0.532241,17.332958,80,0.000000,1,1,13.895475,17.332958,12.753460,18.011205
3,0.586506,13.386359,2,0.000000,0,0,14.012659,13.386359,17.332958,0.000000
4,0.586506,11.377793,0,0.342105,1,1,14.255779,12.723426,0.000000,17.323551
...,...,...,...,...,...,...,...,...,...,...
13413,0.586506,13.564794,2,0.820809,1,1,14.255779,12.688738,12.098791,17.317810
13414,0.586506,14.414472,5,0.820809,0,0,13.802683,12.847837,12.688738,17.315139
13415,0.586506,12.565813,1,0.820809,0,0,13.802683,12.847837,12.688738,17.315139
13416,0.586506,11.964511,0,0.820809,0,1,13.802683,12.847837,12.688738,17.315139


# Regresion

In [640]:
import sklearn as sk
import sklearn.neural_network


lr = 0.001           # learning rate
nn = [2, 16, 8, 1]  # número de neuronas por capa.

# Creamos el objeto del modelo de red neuronal multicapa.
modelRegresor = sk.neural_network.MLPRegressor(solver='adam', 
                                     activation = 'logistic',
                                     learning_rate_init=lr,
                                     hidden_layer_sizes=tuple(nn[1:]),
                                     verbose=True,
                                     n_iter_no_change=1000,
                                     batch_size = 64)

In [641]:
modelRegresor.fit(X_train, y_train)

Iteration 1, loss = 0.12469581
Iteration 2, loss = 0.12232364
Iteration 3, loss = 0.11518507
Iteration 4, loss = 0.10069316
Iteration 5, loss = 0.09194928
Iteration 6, loss = 0.08629797
Iteration 7, loss = 0.08148674
Iteration 8, loss = 0.07718641
Iteration 9, loss = 0.07261138
Iteration 10, loss = 0.06987237
Iteration 11, loss = 0.06833013
Iteration 12, loss = 0.06754850
Iteration 13, loss = 0.06679578
Iteration 14, loss = 0.06662670
Iteration 15, loss = 0.06631772
Iteration 16, loss = 0.06600736
Iteration 17, loss = 0.06578713
Iteration 18, loss = 0.06550743
Iteration 19, loss = 0.06537931
Iteration 20, loss = 0.06519059
Iteration 21, loss = 0.06523537
Iteration 22, loss = 0.06514593
Iteration 23, loss = 0.06501282
Iteration 24, loss = 0.06465961
Iteration 25, loss = 0.06469197
Iteration 26, loss = 0.06474795
Iteration 27, loss = 0.06470627
Iteration 28, loss = 0.06437961
Iteration 29, loss = 0.06435943
Iteration 30, loss = 0.06424353
Iteration 31, loss = 0.06428105
Iteration 32, los

MLPRegressor(activation='logistic', batch_size=64,
             hidden_layer_sizes=(16, 8, 1), n_iter_no_change=1000,
             verbose=True)

In [642]:
LinReg_pred = modelRegresor.predict(X_test)

In [643]:
probando = pd.DataFrame()
probando['target'] = LinReg_pred
probando.describe()

,target
count,3354.000000
mean,0.532715
std,0.298052
min,-0.012716
25%,0.249113
50%,0.518282
75%,0.810015
max,1.012100


In [644]:
modelRegresor.score(X_test, y_test)

0.18079451265763824

In [645]:
loss = log_loss(y_test  , LinReg_pred)
print(f"Log loss is {loss}")

Log loss is 0.5745207563915581


In [646]:
#pickle.dump(modelRegresor, open('modeloRegresorRN.p', 'wb'))

# Kaggle

In [647]:
DataFrame_test = pd.read_csv( "/home/bautista/Datos/Machine-Learning-Datos/FeatureEngineering/df_time_series_test.csv" )
DataFrame_test

,Unnamed: 0,Opportunity_ID,Region,Total_Amount,TRF,Delivery_Year,Opportunity_Owner,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
0,171,10792,EMEA,12.087389,1,2019,Person_Name_13,Q1,1,1,11.753909,11.001991,NaN,11.000704
1,226,10806,EMEA,10.325097,0,2019,Person_Name_13,Q1,0,1,11.753909,11.001991,NaN,11.000704
2,238,10812,EMEA,11.479095,0,2019,Person_Name_13,Q1,1,1,11.753909,11.001991,NaN,11.000704
3,270,10836,EMEA,10.155219,0,2019,Person_Name_13,Q1,1,1,11.753909,11.001991,NaN,11.000704
4,282,10845,EMEA,9.975017,0,2019,Person_Name_13,Q1,1,1,11.753909,11.001991,NaN,11.000704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,2519,12347,Americas,15.340732,12,2020,Person_Name_9,Q2,1,1,13.560399,15.849836,2.081695,15.849825
1563,2520,12348,Americas,16.193551,28,2020,Person_Name_9,Q2,1,1,13.560399,15.849836,2.081695,15.849825
1564,2521,12349,Americas,15.831542,20,2020,Person_Name_9,Q2,1,1,13.560399,15.849836,2.081695,15.849825
1565,2522,12350,Americas,16.193551,28,2020,Person_Name_9,Q3,1,1,14.756877,16.193551,2.081695,16.193543


In [648]:
subir = pd.DataFrame()
subir['Opportunity_ID'] = DataFrame_test['Opportunity_ID']
DataFrame_test = DataFrame_test.drop(columns = {'Opportunity_ID', 'Unnamed: 0', 'Delivery_Year', 'Delivery_Quarter'}).fillna(0)
DataFrame_test

,Region,Total_Amount,TRF,Opportunity_Owner,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Promedio_Region_Por_Quarter,Promedio_Owner_Por_Year_And_Quarter,Lag_1,Delta
0,EMEA,12.087389,1,Person_Name_13,1,1,11.753909,11.001991,0.000000,11.000704
1,EMEA,10.325097,0,Person_Name_13,0,1,11.753909,11.001991,0.000000,11.000704
2,EMEA,11.479095,0,Person_Name_13,1,1,11.753909,11.001991,0.000000,11.000704
3,EMEA,10.155219,0,Person_Name_13,1,1,11.753909,11.001991,0.000000,11.000704
4,EMEA,9.975017,0,Person_Name_13,1,1,11.753909,11.001991,0.000000,11.000704
...,...,...,...,...,...,...,...,...,...,...
1562,Americas,15.340732,12,Person_Name_9,1,1,13.560399,15.849836,2.081695,15.849825
1563,Americas,16.193551,28,Person_Name_9,1,1,13.560399,15.849836,2.081695,15.849825
1564,Americas,15.831542,20,Person_Name_9,1,1,13.560399,15.849836,2.081695,15.849825
1565,Americas,16.193551,28,Person_Name_9,1,1,14.756877,16.193551,2.081695,16.193543


# Encoding

In [649]:
DataFrame_test['Region'] =  DataFrame_test['Region'].map(Mean_encoded_subject_region) 

#DataFrame_test['Delivery_Quarter'] =  DataFrame_test['Delivery_Quarter'].map(Mean_encoded_subject_quarter)

DataFrame_test['Opportunity_Owner'] =  DataFrame_test['Opportunity_Owner'].map(Mean_encoded_subject_owner)
DataFrame_test = DataFrame_test.fillna(0.406068)

In [650]:
DataFrame_test.isnull().sum()

Region                                 0
Total_Amount                           0
TRF                                    0
Opportunity_Owner                      0
Pricing, Delivery_Terms_Approved       0
Pricing, Delivery_Terms_Quote_Appr     0
Promedio_Region_Por_Quarter            0
Promedio_Owner_Por_Year_And_Quarter    0
Lag_1                                  0
Delta                                  0
dtype: int64

# Prediction

In [651]:
pred_posta = modelRegresor.predict(DataFrame_test)

In [652]:
subir['target'] = pred_posta
subir.set_index('Opportunity_ID', inplace = True)
subir

,target
Opportunity_ID,
10792,0.431002
10806,0.474087
10812,0.604836
10836,0.730989
10845,0.748414
...,...
12347,0.157009
12348,0.032578
12349,0.091384


In [653]:
subir.describe()

,target
count,1567.000000
mean,0.364428
std,0.193989
min,-0.001015
25%,0.222963
50%,0.304910
75%,0.482467
max,1.015769


In [654]:
#subir.to_csv('RedesNeuronales_Entrega_8.csv')